In [0]:
%tensorflow_version 2.x
import tensorflow as tf

# layers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
# model
from tensorflow.keras.models import Model

# dataset
from tensorflow.keras.datasets import cifar10

TensorFlow 2.x selected.


In [0]:
# additional imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# load the data
cifar10_dataset = cifar10.load_data() # this'll return a tuple, first element contains training data and the second one contains test data
(x_train, y_train), (x_test, y_test) = cifar10_dataset

# look at the shape of the data
print('Shape of the X: \n Test: {}, Train: {}'.format(x_test.shape, x_train.shape))
print('Shape of the Y: \n Test: {}, Train: {}'.format(y_test.shape, y_train.shape))

170500096/170498071 [==============================] - 6s 0us/step
Shape of the X: 
 Test: (10000, 32, 32, 3), Train: (50000, 32, 32, 3)
Shape of the Y: 
 Test: (10000, 1), Train: (50000, 1)


In [0]:
# we don't need to process the X because its already 3D.
# but we need to flatten the Y
y_train, y_test = y_train.flatten(), y_test.flatten()
print('Shape of the Y: \n Test: {}, Train: {}'.format(y_test.shape, y_train.shape))

Shape of the Y: 
 Test: (10000,), Train: (50000,)


In [0]:
# X contains the pixel values of images, these values are in the range of 0,255
# i'd like to centerize my data if i can because lower variance means more uniformly distributed data. So i'll divide all the values by 255, this will reduce the range to 0,1
print(x_train.std(), x_train.mean())
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.std(), x_train.mean())

64.1500758911213 120.70756512369792
0.25156892506322026 0.4733630004850874


In [0]:
x_train[0].shape

(32, 32, 3)

In [0]:
# Now we can build the model using functional api
i = Input(shape = x_train[0].shape)
x = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x) # Dense layer'ı input olarak 1D array alır
x = Dropout(.4)(x) # to reduce overfitting
x = Dense(512, activation = 'relu')(x)
x = Dropout(.2)(x) # to reduce overfitting
x = Dense(10, activation = 'softmax')(x)

model = Model(i, x)

NameError: ignored

In [0]:
# compile the model
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [0]:
# fit the model
r = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50)

In [0]:
# data augmention
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=.1, height_shift_range = .1, horizontal_flip = True)
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
r = model.fit_generator(train_generator, validation_data = (x_test, y_test), steps_per_epoch=steps_per_epoch, epochs = 50)

In [0]:
# plot the results

# loss
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [0]:
# accuracy
plt.plot(r.history['acc'], label = 'accuracy')
plt.plot(r.history['val_acc'], label = 'val_accuracy')
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting 'normalize=True'.
  """
  if normalize:
    cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')
  print(cm)

  plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment='center',
             color = 'white' if cm[i, j] > thresh else 'black')
  plt.tight_layout()
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label')
  plt.show()

p_test = model.predict(x_test).argmax(axis = 1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [0]:
# summary of model
model.summary()

In [0]:
labels = ['uçak', 'otomobil', 'kuş', 'kedi', 'geyik', 'köpek', 'kurbağa', 'at', 'gemi', 'kamyon']
# random false predictions
misclsf_idx = np.where(p_test != y_test)[0]
random_element = np.random.choice(misclsf_idx)
plt.imshow(x_test[random_element])
plt.title('Tahmin edilen: {} | Gerçekte olan: {}'.format(labels[p_test[random_element]], labels[y_test[random_element]]))